## 1. mecab설치 및 사용자 사전 적용


In [1]:
!pip install cdqa
!pip install konlpy

     |████████████████████████████████| 51kB 5.5MB/s 
     |████████████████████████████████| 102kB 8.2MB/s 
     |████████████████████████████████| 286kB 10.9MB/s 
     |████████████████████████████████| 10.4MB 24.7MB/s 
     |████████████████████████████████| 317kB 45.5MB/s 
     |████████████████████████████████| 6.7MB 52.9MB/s 
     |████████████████████████████████| 748.9MB 17kB/s 
     |████████████████████████████████| 92kB 10.9MB/s 
     |████████████████████████████████| 51kB 7.8MB/s 
     |████████████████████████████████| 890kB 45.7MB/s 
     |████████████████████████████████| 133kB 57.6MB/s 
     |████████████████████████████████| 1.2MB 45.6MB/s 
     |████████████████████████████████| 81kB 13.0MB/s 
     |████████████████████████████████| 7.3MB 47.8MB/s 
  Created wheel for cdqa: filename=cdqa-1.3.9-cp37-none-any.whl size=47640 sha256=422054519e696e08d7d0740caa673759d9e9871bc401823cf1474a2aac826594
  Stored in directory: /root/.cache/pip/wheels/8b/9a/68/d3f7651ea29c30d1beb

     |████████████████████████████████| 19.4MB 32.4MB/s 
     |████████████████████████████████| 92kB 10.6MB/s 
     |████████████████████████████████| 460kB 45.6MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
import os
from google.colab import drive
import pandas as pd
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
# mecab 재설정
os.chdir('/content/gdrive/My Drive/colabnotebooks/bert_folder/kor/KorBERT-cdQA/지우지마/mecab-python-0.996')
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
!python3 setup.py build
!python3 setup.py install
os.chdir("/content/gdrive/My Drive/colabnotebooks/bert_folder/kor/KorBERT-cdQA/result")

Installing automake (A dependency for mecab-ko)
Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [1,748 kB]
Ign:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86

FileNotFoundError: ignored

In [ ]:
import pandas as pd
os.chdir("/content/gdrive/My Drive/colabnotebooks/bert_folder/kor/KorBERT-cdQA/result")
dic = pd.read_csv('./disease_dic.csv',encoding="euc-kr")
os.chdir("/content/gdrive/My Drive/colabnotebooks/bert_folder/kor/KorBERT-cdQA/result/tmp/mecab-ko-dic-2.1.1-20180720/user-dic/")
pd.concat([pd.read_csv("nnp.csv"),dic],axis=0).to_csv("nnp.csv",index=False)

In [ ]:
os.chdir("/tmp/mecab-ko-dic-2.1.1-20180720")
# 새로 넣은 dic.csv를 사전에 추가
! ./tools/add-userdic.sh
# 새로운 csv 파일을 사전에 적용및 설치
! make clean
! make install

In [ ]:
os.chdir("/content/gdrive/My Drive/colabnotebooks/bert_folder/kor/KorBERT-cdQA/result")

## 2. 데이터 전처리

In [ ]:
from KorBERT import ETRI_korBERT,ETRI_POS_Tagging,ETRI_wiki,Pos_extract
from cdqa.retriever import BM25Retriever,TfidfRetriever
import konlpy 
from konlpy.tag import Mecab
import pandas as pd
import numpy as np
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt
import os
import sys
import re
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
pos_tagger = Mecab()

In [ ]:
def cleantext(text):
  enter = re.compile("[^가-힣|A-z]+")
  clean_text = enter.sub(" ",text)
  clean_text = re.sub("국가건강정보포털"," ",clean_text)
  clean_text = re.sub("서울대학교병원"," ",clean_text)
  clean_text = re.sub("저작권법"," ",clean_text)
  clean_text = re.sub("콘텐츠"," ",clean_text)
  clean_text = re.sub("무단"," ",clean_text)
  clean_text = re.sub("https://www.naver.com"," ",clean_text)
  clean_text = re.sub("tv"," ",clean_text)
  clean_text = clean_text.strip()
  return clean_text

def stop_word(text):
  result = []
  a = pos_tagger.morphs(text)
  stop_word = [
              "으로","개월","습니다","하지만","이나","또는",
              "다만","습니다","심지어","합니다","관련","이러",
              "으며","경우","라는","이를","입니다","다행히",
              "시켜","에서","경우","인해","라는","됩니다",
              "지만","그렇","다고","해서","그러므로","통해",
              "특히","거의","없이","거나","시킬","대한",
              "처럼","따른","시키","비해","아닙니다","다면",
              "라고","으므로","해야","그러나","이런","대해서",
              "으므로","함","으로써","이루어지","아닙니다","에게",
              "통한","그리고","다는","는데","으나","인한",
              "집니다","때문","이후","크","나타나","만큼",
              "의한","의해서","대개","또한","도록","관해",
              "해준다","이르","므로","따라","때로","위해",
              "주후","대로","따라서","다가","아야","이때",
              "아니","물론","계시","대단히","오직","계시",
              "어쩔","이룰","무척","똑같이","실제로","그래서",
              "왜냐하면","바랍니다","비로소","에선","가령","빈번히",
              "이른바","라면","아니","못한","대단히","철저히",
              "물론","네이버","그것","중요","요즘","그때",
              "무엇","이제","해당","한동안","예전","주지",
              "뭔가","듯이","약간","도중","계속","의학",
              "정보","듭니다","간혹","에서부터","의해","한다",
              "에게서","혹은","아주","일부","결국","빠질",
              "생긴다","대체","그러","제대로","너무","된다",
              "로부터","는지","까지","주로","면서","이내",
              "더라도","보인다면","보다","아직","위한","우리",
              "는다","인데","어야","어서","정도","한다고",
              "nan","어요","생겨","생긴","기타","졌어요",
              "어떤","못하","보이","해짐","여러","나타남",
              "갑자기","대하","어도","동안","떠올라","때때로",
              "서로","내게","양쪽","한쪽"
              ]
  non_stop_word = ["낮","밤",
                  "뇌","팔","눈","코","입","귀","혀","손","배","등","뼈","골","목","속","턱"
                  "물","불","간","똥","멍","위","털","않","변","맥","돌",
                  "말","욕","암","성","형","병","증","콩","해","토","술","칼","화","열","몸","안"]
  for i in a:
    if i not in stop_word:
      if len(i) != 1:
        result.append(i)
      elif i in non_stop_word:
        result.append(i)
  return result

## 3. 데모 버전

In [ ]:
dff = pd.read_csv("./result_v2.csv")
def get_retriever():
  retriever = BM25Retriever(lowercase=False,
                            tokenizer = stop_word
                            )
  return retriever
retriever = get_retriever()
retriever.fit(dff)

In [ ]:
query1 = input("증상을 적어주세요. : ")
query1 = " ".join(stop_word(query1))
top_5 = list(retriever.predict(query1).keys())[:5]
print("-"*100)
print(dff.iloc[top_5,[0,1,2]])